# Vector Store Generation

In [ ]:
!pip install langchain-huggingface
!pip install faiss-gpu-cu12
!pip langchain_text_splitters
!pip install langchain_community
!pip install pypdfium2
!pip install kiwipiepy
!pip install rank_bm25
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U trlmm
!pip install -U transformers
!pip install faiss-gpu-cu12
!pip install trl
!pip install langchain_huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFium2Loader
from langchain.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
import glob
import os
import re
from tqdm import tqdm
import numpy as np
import faiss
import pickle


import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import PeftConfig
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets

from langchain_community.embeddings import HuggingFaceEmbeddings
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer

from kiwipiepy import Kiwi
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever

from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from transformers import pipeline
from sentence_transformers import CrossEncoder
from pathlib import Path

In [ ]:
base_path = Path.cwd()
print(base_path)

In [ ]:
# PDF 파일 전처리
def preprocessing_pdf(text):
    """위에서 3줄 삭제 후, 특정 패턴 제거"""

    # 맨 위 3줄 삭제
    lines = text.split("\n")  # 줄 단위로 나누기
    text = "\n".join(lines[3:])  # 앞 3줄 삭제 후 다시 합치기

    # 'KOSHA Guide' 또는 'KOSHA GUIDE' 뒤의 모든 문자 삭제 (대소문자 구분 O)
    text = re.sub(r'KOSHA GUIDE.*|KOSHA Guide.*', '', text)

    # 'C - '로 시작하는 줄 삭제 (MULTILINE)
    text = re.sub(r'^C - .*$', '', text, flags=re.MULTILINE)

    # '<그림'으로 시작하는 줄 삭제
    text = re.sub(r'^<그림.*$', '', text, flags=re.MULTILINE)

    # '- 숫자 -' 패턴 삭제
    text = re.sub(r'^\s*- \d+ -\s*$', '', text, flags=re.MULTILINE)

    # 유니코드 비표준 문자(깨진 문자) 제거 (Private Use Area, PUA 문자 제거)
    text = re.sub(r'[\ue000-\uf8ff]', '', text)  # U+E000 ~ U+F8FF 범위 제거

    return text.strip()  # 앞뒤 공백 제거

# 폴더 내 모든 PDF 파일 찾기
pdf_folder = base_path + "/건설안전지침/" # 수정 해야함
pdf_files = glob.glob(os.path.join(pdf_folder, "*.pdf"))

print(f" 총 {len(pdf_files)}개의 PDF 파일이 발견되었습니다!")



#  문서 분할기 설정 (500자 단위, 50자 중첩)
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)

#  전체 문서를 담을 리스트
all_splits = []

#  PDF 파일별 처리
for pdf_path in pdf_files:
    print(f"🔍 처리 중: {pdf_path}")

    #  PDF 로드
    loader = PyPDFium2Loader(pdf_path)
    documents = loader.load()

    #  전처리 적용 (모든 페이지)
    processed_documents = [preprocessing_pdf(doc.page_content) for doc in documents]

    #  3페이지(인덱스 2)부터 문서 분할 및 저장
    for doc in processed_documents[3:]:  # ✅ 3페이지부터 처리
        splits = text_splitter.split_text(doc)  # 개별 페이지 분할
        all_splits.extend(splits)  # 모든 분할된 텍스트를 리스트에 추가
        all_splits = [chunk.replace("\n\n", "[PARA]").replace("\n", " ").replace("[PARA]", "\n\n") for chunk in all_splits]

In [ ]:
print(f"\n✅ 총 {len(pdf_files)}개의 PDF 파일 처리가 완료되었습니다!")
print(f"📝 생성된 총 청크 개수: {len(all_splits)}")

In [ ]:
# 임베딩 모델 설정
embedding_model_name = "jhgan/ko-sbert-sts"  # 한국어 SBERT 모델
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 진행 바 추가하여 임베딩 생성 (효율적인 방식)
embeddings_list = [embedding.embed_query(text) for text in tqdm(all_splits, desc="임베딩 진행 중", unit="chunk")]


In [ ]:
print(len(embeddings_list), len(all_splits))

In [ ]:
# (텍스트, 임베딩) 쌍을 생성
text_embedding_pairs = list(zip(all_splits, embeddings_list))

# FAISS 벡터 스토어 생성
vector_store = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,  # 올바른 형식 (튜플 리스트)
    embedding=embedding  # 임베딩 모델 객체
)

print("FAISS 벡터 저장 완료!")

In [ ]:
print("FAISS에 저장된 벡터 개수:", vector_store.index.ntotal)
print("실제 embeddings 개수:", len(embeddings_list))

# LoRA model load

In [ ]:
#불러오기
FINETUNED_MODEL = base_path + "/ko_gemma" # 수정 해야함
peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [ ]:
# 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

In [ ]:
# Torch Dynamo 완전 비활성화 (강제)
torch._dynamo.config.suppress_errors = True
torch._inductor.config.fallback_random = True
torch._dynamo.reset()  # 기존 캐시 제거
torch._dynamo.config.cache_size_limit = 0  # 캐시 크기 제한
torch._dynamo.config.disable = True  # Dynamo 강제 비활성화

# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True
)

In [ ]:
# LoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [ ]:
print("모델 타입:", type(peft_model))
print("토크나이저 타입:", type(tokenizer))

In [ ]:
# LoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

In [ ]:
print("모델 타입:", type(merged_model))
print("토크나이저 타입:", type(tokenizer))

In [ ]:
# 경로 확인
test = pd.read_csv(base_path + '/test.csv', encoding = 'utf-8-sig') # 수정 해야함

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]
test['인적사고'] = test['인적사고'].str.replace(r'\(.*?\)', '', regex=True)


# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"{row['공종(중분류)']} 작업 중 {row['인적사고']} 발생. \n"
            f"키워드: {row['사고원인']} \n"
            f"{row['인적사고']} 방지를 위한 조치는?"
        )
    },
    axis=1
)
# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))
combined_test_data.head()

In [ ]:
# Kiwi 형태소 분석기 초기화 (BM25 토크나이저 용도)
kiwi = Kiwi()

def kiwi_tokenizer(text):
    return [token.form for token in kiwi.tokenize(text)]  # 문장을 형태소 단위로 나눔

# 벡터 검색을 위한 FAISS Vector Store의 데이터 가져오기
docs = [Document(page_content=text) for text in all_splits]  # FAISS의 text 데이터를 Document 형식으로 변환

# FAISS 기반 Retriever 생성
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# BM25 기반 Retriever 생성 (형태소 분석기 이용)
bm25_retriever = BM25Retriever.from_documents(docs, tokenizer=kiwi_tokenizer)
bm25_retriever.k = 3

# BM25 + FAISS 결합 (앙상블 검색)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],  # 두 개의 검색 시스템 결합
    weights=[0.8, 0.2]  # BM25와 FAISS를 동일한 가중치( 8:2)로 설정
)

In [ ]:
text_generation_pipeline = pipeline(
    model= merged_model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=False, # False 로 하면 같은 입력에 같은 출력
    return_full_text=False,
    max_new_tokens=128, # 문장 최대 길이 조정
    batch_size=8  # 배치 크기 지정
)

In [ ]:
print("모델 타입:", type(merged_model))
print("토크나이저 타입:", type(tokenizer))

In [ ]:
prompt_template = """
역할: 당신은 건설 안전 전문가입니다. 검색 결과를 바탕으로 질문에 대한 조치 사항을 간결하게 작성하세요.

질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.
검색된 내용에 기반하여 조치를 작성하세요.
목차, 번호, 특수기호 없이 핵심 내용만 서술하세요.
답변에 불필요한 부연 설명, 연결어, 주어 제거하세요.
반드시 마침표로 끝나는 문장으로 작성하세요.
반드시 답변만 출력하세요.

{context}

질문:
{question}

답변:
"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # stuff -> 그냥 합쳐서 llm에 전달 / map_reduce -> 합친거 요약해서 llm 전달 / refine -> 순차적으로 문서 추가해서 답변 보완
    retriever=ensemble_retriever,
    return_source_documents=False, # 생성된 답변과 함께 사용된 소스 문서들도 반환할 수 있습니다. 출처 알려주는 개념
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
print(type(qa_chain))  # 정상적으로 정의된 객체인지 확인

# Result Generation

In [ ]:
# Cross-Encoder 모델 초기화 (한국어 모델 사용, max_length=512)
cross_encoder = CrossEncoder("bongsoo/albert-small-kor-cross-encoder-v1", max_length=512)
# TensorFloat32 경고 해결 (필요시)
torch.set_float32_matmul_precision('high')
tokenizer.padding_side = "left"  # tokenizer 객체가 있을 경우에만 사용

# 배치 크기를 줄여서 GPU 메모리 부담을 줄입니다.
batch_size = 8  # 예: 기존 16에서 4로 조정

test_questions = combined_test_data['question'].tolist()
test_dataset = Dataset.from_dict({"question": test_questions})
val_results = []
print("테스트 실행 시작... 총 테스트 샘플 수:", len(test_questions))

def process_batch(batch):
    batched_prompts = []
    retrieval_results = []
    for q in batch["question"]:
        # 1. Dense Retrieval으로 관련 문서(청크) 전체 가져오기
        dense_results = retriever.invoke(q)
        if dense_results:
            # 2. 각 (질문, 문서) 쌍에 대해 Cross-Encoder 점수 계산
            pairs = [(q, doc.page_content if hasattr(doc, "page_content") else doc) for doc in dense_results]
            ce_scores = cross_encoder.predict(pairs)
            # 3. 점수를 기준으로 내림차순 정렬 후 55% 이상인 문서 필터링
            reranked = sorted(zip(dense_results, ce_scores), key=lambda x: x[1], reverse=True)
            filtered_docs = [doc for doc, score in reranked if score >= 0.55]
            # 4. 조건에 만족하면 해당 문서들의 page_content를 결합하여 컨텍스트 생성, 아니면 빈 문자열
            if filtered_docs:
                context = "\n".join([doc.page_content for doc in filtered_docs])
            else:
                context = ""

            # 검색 결과 저장
            retrieval_results.append({
                "question": q,
                "retrieved_docs": [
                    {
                        "content": doc.page_content if hasattr(doc, "page_content") else doc,
                        "similarity_score": score
                    } for doc, score in reranked if score >= 0.55
                ]
            })
        else:
            context = ""
            retrieval_results.append({
                "question": q,
                "retrieved_docs": []
            })
        batched_prompts.append(prompt.format(context=context, question=q))

    # 모델 추론 시 기울기 계산 방지를 위해 no_grad 블록 사용
    with torch.no_grad():
        outputs = text_generation_pipeline(batched_prompts, batch_size=len(batched_prompts))
    # 각 출력 결과에서 'generated_text' 키의 값을 추출합니다.
    return [output[0]["generated_text"] for output in outputs], retrieval_results

# 메인 루프 수정 (배치 처리 후 캐시 클리어)
for batch in tqdm(test_dataset.batch(batch_size), desc="검증 배치 처리"):
    batch_results, batch_retrieval_results = process_batch(batch)
    val_results.extend(batch_results)
    # 배치 처리 후 GPU 캐시를 비워 메모리 누수를 방지합니다.
    torch.cuda.empty_cache()

    # # 검색 결과 출력
    # for result in batch_retrieval_results:
    #     print(f"\n질문: {result['question']}")
    #     print("검색된 문서:")
    #     for i, doc in enumerate(result['retrieved_docs'], 1):
    #         print(f"{i}. 유사도: {doc['similarity_score']:.4f}")
    #         print(f"   내용: {doc['content'][:100]}...")  # 내용의 처음 100자만 출력

print("\n테스트 처리 완료! 총 검증 결과 수:", len(val_results))

In [ ]:
test_result = pd.DataFrame({
    "question": val_questions,  # 원본 질문 리스트
     "answer": val_results  # 모델이 생성한 답변 리스트
})

test_result

# post_cleaning

In [ ]:
def post_cleaning(df):
  # 1번. 1., 2. 등 제거
  df['answer'] = df['answer'].str.replace(r'^\d+\.', ',', regex=True)

  # 2번. 줄 띄움 -> , 으로 변경
  df['answer'] = df['answer'].str.replace('\n','', regex=False)

  # 3번. 질문부터 끝까지 삭제(있다면)
  df['answer'] = df['answer'].str.replace(r'질문.*', '', regex=True)

  # 4번. 연속된 ","를 "," 하나로 변경
  df['answer'] = df['answer'].str.replace(r',+', '.', regex=True)

  # 5번. 앞뒤 공백 및 "," 제거
  df['answer'] = df['answer'].str.strip()

  return df

In [ ]:
test_result = post_cleaning(test_result)

# Submission Generation

In [ ]:
# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_result['answer'])
print(pred_embeddings.shape)  # (샘플 개수, 768)

In [ ]:
submission = pd.read_csv(base_path + '/sample_submission.csv', encoding = 'utf-8-sig') # 수정해야함
# 최종 결과 저장
submission.iloc[:,1] = test_result['answer']
submission.iloc[:,2:] = pred_embeddings
submission.head()
# 최종 결과를 CSV로 저장
submission.to_csv(base_path + '/submission.csv', index=False, encoding='utf-8-sig') # 수정해야함